## Importing all packages

In [32]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from matplotlib import pyplot as plt
import openpyxl
import numpy as np

## Opioid Dispensing Rate
### - Scraping one table to get code working

In [33]:
website_url= 'https://www.cdc.gov/drugoverdose/maps/rxstate2019.html'
response = requests.get(website_url)
response.status_code

200

In [34]:
soup = BeautifulSoup(response.content, 'lxml')
print(soup.title)

<title>U.S. State Opioid Dispensing Rates, 2019  | Drug Overdose | CDC Injury Center </title>


In [35]:
tables = soup.find_all('table', attrs = {'class': 'table table-striped'})  #filters table down even more-drill 
#tables

In [36]:
type(tables)

bs4.element.ResultSet

In [37]:
len(tables)

1

In [38]:
result_list = pd.read_html(str(tables[0])) # a list of dataframes
len(result_list)

1

In [39]:
type(result_list)

list

In [40]:
world_soups = result_list[0]  # get the first df from the list
world_soups.head()

,State,Abbreviation,Opioid Dispensing Rate per 100
0,United States,US,46.7
1,Alaska,AK,39.1
2,Alabama,AL,85.8
3,Arkansas,AR,80.9
4,Arizona,AZ,44.1


In [41]:
world_soups['Year']=2019

In [42]:
world_soups.head()

,State,Abbreviation,Opioid Dispensing Rate per 100,Year
0,United States,US,46.7,2019
1,Alaska,AK,39.1,2019
2,Alabama,AL,85.8,2019
3,Arkansas,AR,80.9,2019
4,Arizona,AZ,44.1,2019


### - Scraping all years 

In [43]:
all_years = pd.DataFrame()
for year in range (2006,2020):
    cdc_url = f'https://www.cdc.gov/drugoverdose/maps/rxstate{year}.html'
    response = requests.get(cdc_url)
    soup = BeautifulSoup(response.content, 'lxml')

    tables = soup.find_all('table', attrs = {'class': 'table table-striped'})
    result_list = pd.read_html(str(tables[0]))
    world_soups = result_list[0]
    world_soups['Year']=year

    all_years=all_years.append(world_soups)

In [44]:
all_years.head()

,State,State Abbreviation,Opioid Dispensing Rate per 100,Year,Abbreviation
0,Alabama,AL,115.6,2006,NaN
1,Alaska,AK,63.4,2006,NaN
2,Arizona,AZ,74.3,2006,NaN
3,Arkansas,AR,98.3,2006,NaN
4,California,CA,51.0,2006,NaN


In [45]:
len(all_years)

717

In [46]:
## taking out NaN and making one column with state abbreviation
all_years['my_state']=np.nan
all_years['my_state']= all_years.my_state.fillna(all_years['State Abbreviation']).fillna(all_years.Abbreviation)


In [47]:
all_years

,State,State Abbreviation,Opioid Dispensing Rate per 100,Year,Abbreviation,my_state
0,Alabama,AL,115.6,2006,NaN,AL
1,Alaska,AK,63.4,2006,NaN,AK
2,Arizona,AZ,74.3,2006,NaN,AZ
3,Arkansas,AR,98.3,2006,NaN,AR
4,California,CA,51.0,2006,NaN,CA
...,...,...,...,...,...,...
47,Vermont,NaN,36.9,2019,VT,VT
48,Washington,NaN,42.7,2019,WA,WA
49,Wisconsin,NaN,42.5,2019,WI,WI
50,West Virginia,NaN,59.4,2019,WV,WV


In [48]:
#Taking out extra columns
all_years=all_years[['State','Opioid Dispensing Rate per 100','Year','my_state']]

In [50]:
all_years

,State,Opioid Dispensing Rate per 100,Year,my_state
0,Alabama,115.6,2006,AL
1,Alaska,63.4,2006,AK
2,Arizona,74.3,2006,AZ
3,Arkansas,98.3,2006,AR
4,California,51.0,2006,CA
...,...,...,...,...
47,Vermont,36.9,2019,VT
48,Washington,42.7,2019,WA
49,Wisconsin,42.5,2019,WI
50,West Virginia,59.4,2019,WV


## Bringing in Cause of Death by state data
### - Imported openpyxl above to load in an xlsx file
### - Data is for dates 1999-2019

In [62]:
# reading in data, excluding notes at bottom (after ros 11831)
all_years_cod_df = pd.read_excel('../data/Indiviual_years_Overdose.xlsx',nrows=11831, sheet_name = 0)

In [63]:
all_years_cod_df

,Notes,State,State Code,Year,Multiple Cause of death,Multiple Cause of death Code,Deaths,Population
0,NaN,Alabama,1,1999.0,Opium,T40.0,Suppressed,4430141
1,NaN,Alabama,1,1999.0,Heroin,T40.1,Suppressed,4430141
2,NaN,Alabama,1,1999.0,Other opioids,T40.2,12,4430141
3,NaN,Alabama,1,1999.0,Methadone,T40.3,16,4430141
4,NaN,Alabama,1,1999.0,Other synthetic narcotics,T40.4,11,4430141
...,...,...,...,...,...,...,...,...
11826,NaN,Wyoming,56,2019.0,Other and unspecified narcotics,T40.6,Suppressed,578759
11827,NaN,Wyoming,56,2019.0,Cannabis (derivatives),T40.7,10,578759
11828,NaN,Wyoming,56,2019.0,Lysergide [LSD],T40.8,Suppressed,578759
11829,NaN,Wyoming,56,2019.0,Other and unspecified psychodysleptics [halluc...,T40.9,Suppressed,578759


In [67]:
all_years_cod_df['Year'] = all_years_cod_df['Year'].astype(int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [66]:
all_years_cod_df

,Notes,State,State Code,Year,Multiple Cause of death,Multiple Cause of death Code,Deaths,Population
0,NaN,Alabama,1,1999.0,Opium,T40.0,Suppressed,4430141
1,NaN,Alabama,1,1999.0,Heroin,T40.1,Suppressed,4430141
2,NaN,Alabama,1,1999.0,Other opioids,T40.2,12,4430141
3,NaN,Alabama,1,1999.0,Methadone,T40.3,16,4430141
4,NaN,Alabama,1,1999.0,Other synthetic narcotics,T40.4,11,4430141
...,...,...,...,...,...,...,...,...
11826,NaN,Wyoming,56,2019.0,Other and unspecified narcotics,T40.6,Suppressed,578759
11827,NaN,Wyoming,56,2019.0,Cannabis (derivatives),T40.7,10,578759
11828,NaN,Wyoming,56,2019.0,Lysergide [LSD],T40.8,Suppressed,578759
11829,NaN,Wyoming,56,2019.0,Other and unspecified psychodysleptics [halluc...,T40.9,Suppressed,578759


###  Cause of death for all years 1999-2019 all narcotics

In [4]:
#all_drug_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 1)
#heroin_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 2)
#opioids_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 3)
#methadone_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 4)
#synthetic_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 5)
#cocaine_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 6)
#other_narcs_cod_df = pd.read_excel('../data/Cause_of_death.xlsx', sheet_name = 7)

#### - Dropping columns

In [5]:
all_drug_cod_df=all_drug_cod_df[['State','Deaths','Population']]
heroin_cod_df = heroin_cod_df[['State','Deaths','Population']]
opioids_cod_df = opioids_cod_df[['State','Deaths','Population']]
methadone_cod_df =methadone_cod_df[['State','Deaths','Population']]
synthetic_cod_df = synthetic_cod_df[['State','Deaths','Population']]
cocaine_cod_df = cocaine_cod_df[['State','Deaths','Population']]
other_narcs_cod_df = other_narcs_cod_df[['State','Deaths','Population']]

In [51]:
all_years_cod_df.to_csv("CAUSE_OF_DEATH.csv")